<a href="https://colab.research.google.com/github/jPruim/cs344/blob/master/finalproject/Linear_Algebra_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proposal:
Use neural networks to try and solve common and less common linear algebra problems. This will involve using neural nets and CNN's to try and see how few nodes are necessary to solve some problems.

Specifically I hope to investigate determinants, eigenvalues, and inverses for matrices. Also, I hope to be able to attempt to do a decomposition. Part of the project is determining whether or not it is possible to do some of these approximations.

In [0]:
import numpy as np
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Dropout
import numpy.linalg as linalg

In [72]:
#create datasets, normalized arrays of n^2 size.
#Thus all values are between -1 and 1
#No third set needed as the sets are random and different each time anyway
n = 7
train_set_size = 40000
train_set=np.random.random(size=(train_set_size, n, n))*2-1

val_set_size = 10000
val_set=np.random.random(size=(val_set_size, n, n))*2-1

#print shape
print(train_set[:1])
print(len(train_set))



#Shaped for simple neural networks
train_set2 = np.empty((train_set_size,n**2))
for i in range(train_set_size):
  train_set2[i] = np.ravel(train_set[i])
train_set2.reshape(train_set_size,n**2,1)
print(train_set2.shape)

val_set2 = np.empty((val_set_size,n**2))
for i in range(val_set_size):
  val_set2[i] = np.ravel(val_set[i])
val_set2.reshape(val_set_size,n**2,1)
print(val_set2.shape)

#Shaped for CNN neural networks
train_set3 = train_set.reshape(train_set_size,n,n,1)
val_set3 = val_set.reshape(val_set_size,n,n,1)


[[[-0.04373091 -0.61745869 -0.12409737 -0.74107924 -0.83637785
    0.32079463 -0.25021075]
  [ 0.07600896  0.82222931  0.40687249  0.17299387 -0.971019
   -0.03378352 -0.94829831]
  [-0.38450047 -0.64311001 -0.06316817  0.01006581 -0.70171439
    0.80355225 -0.55963759]
  [-0.9163142   0.87334135 -0.18536459 -0.69522956 -0.69123154
    0.48754666 -0.4216813 ]
  [ 0.75556176  0.67123854 -0.073078    0.97946388  0.14325928
   -0.39541995 -0.81288345]
  [-0.32863349 -0.55992979 -0.17680193  0.44581195 -0.86690555
   -0.12833812 -0.1961278 ]
  [ 0.87907713 -0.24926918  0.27159382 -0.60023955 -0.64500244
   -0.7825694  -0.59805912]]]
40000
(40000, 49)
(10000, 49)


In [73]:
#generate determinant labels
determinant_set_size = train_set_size
determinant_set = np.empty((determinant_set_size,1))
determinant_abs_set  = np.empty((determinant_set_size,1))
for i in range(0,determinant_set_size): 
  determinant_set[i]=linalg.det(train_set[i])
  determinant_abs_set[i] = abs(determinant_set[i])

print(sum(determinant_abs_set) / len(determinant_set))
val_determinant_set_size = val_set_size
val_determinant_set = np.empty((val_set_size,1))
for i in range(0,val_determinant_set_size): 
  val_determinant_set[i]= linalg.det(val_set[i])

print(determinant_set.shape)

[0.93716768]
(40000, 1)


In [74]:
#generate eigenvalue labels
eigenvalue_set_size = train_set_size
eigenvalue_set = np.empty((eigenvalue_set_size,n))
for i in range(0,train_set_size):
  eigenValues= linalg.eigvals(train_set[i])
  eigenvalue_set[i]= eigenValues

val_eigenvalue_set_size = val_set_size
val_eigenvalue_set = np.empty((val_eigenvalue_set_size,n))
for i in range(0,val_eigenvalue_set_size): 
  eigenValues = linalg.eigvals(train_set[i])
  val_eigenvalue_set[i]= eigenValues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


In [75]:
#start modeling.
#determinant modeling 1 with simply ReLu 
#6 computation layers  (and then the final layer)
detmodel = models.Sequential()

detmodel.add(layers.Dense(n**2, activation='relu',input_dim=n**2))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
# model.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(1, activation='relu'))
detmodel.compile(optimizer='adam',
              loss='mean_squared_error')
detmodel.summary()


Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_119 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_120 (Dense)            (None, 50)                2500      
_________________________________________________________________
dense_121 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_122 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_123 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_124 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_125 (Dense)            (None, 1)               

In [76]:
#Simple NN model for determinant with Relu
detmodel.fit(train_set2, determinant_set, epochs=5, batch_size=100)
detmodel.evaluate(val_set2, val_determinant_set)
print(detmodel.predict(val_set2[:10]))
print(val_determinant_set[:10])

Epoch 1/5
40000/40000 [==============================] - 1s 21us/step - loss: 2.3481
Epoch 2/5
40000/40000 [==============================] - 1s 17us/step - loss: 2.3481
Epoch 3/5
40000/40000 [==============================] - 1s 17us/step - loss: 2.3481
Epoch 4/5
40000/40000 [==============================] - 1s 17us/step - loss: 2.3481
Epoch 5/5
10000/10000 [==============================] - 0s 21us/step
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[ 3.73893636]
 [-2.38908039]
 [ 0.09039151]
 [-0.66248686]
 [-0.86043685]
 [ 0.37028895]
 [-0.04389581]
 [-0.88441543]
 [-2.34117211]
 [-0.23658771]]


This simple initial model has a couple worrying features. Most notably, the loss flattens out at a relatively high value. Then with some further investigation, it becomes apparent that the model is only (or almost always) returning 0. This means that the ReLU's are dead.

Some more investigation into Dead ReLU's revealed this:
"Unfortunately, ReLU units can be fragile during training and can "die". For example, a large gradient flowing through a ReLU neuron could cause the weights to update in such a way that the neuron will never activate on any datapoint again. If this happens, then the gradient flowing through the unit will forever be zero from that point on. That is, the ReLU units can irreversibly die during training since they can get knocked off the data manifold. For example, you may find that as much as 40% of your network can be "dead" (i.e. neurons that never activate across the entire training dataset) if the learning rate is set too high. With a proper setting of the learning rate this is less frequently an issue." (CS231 Stanford)



In [78]:
#start modeling.
#determinant 
detmodel2 = models.Sequential()

detmodel2.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3),input_dim=n**2))
detmodel2.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
detmodel2.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
detmodel2.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel2.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
# detmodel2.add(layers.Dense(50, activation='relu'))
# detmodel2.add(layers.Dense(50, activation='relu'))
detmodel2.add(layers.Dense(1, activation=LeakyReLU(alpha=0.5)))
detmodel2.compile(optimizer='adam',
              loss='mean_squared_error')
detmodel2.summary()



Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_132 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_133 (Dense)            (None, 343)               17150     
_________________________________________________________________
dense_134 (Dense)            (None, 343)               117992    
_________________________________________________________________
dense_135 (Dense)            (None, 49)                16856     
_________________________________________________________________
dense_136 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_137 (Dense)            (None, 1)                 50        
Total params: 156,948
Trainable params: 156,948
Non-trainable params: 0
_______________________________________________

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [80]:
#Simple NN model for determinant with Leaky RelU -- Overfits
#The leaky ReLU's solve the Dead ReLU problem.
detmodel2.fit(train_set2, determinant_set, epochs=300, batch_size=1000)

print("\n\nEvaluation:")
print(detmodel2.evaluate(val_set2, val_determinant_set))

print("\n\nManual Comparison on Training data")
print(detmodel2.predict(train_set2[:5]))
print(determinant_set[:5])

print("\n\nManual Comparison on Validation data")
print(detmodel2.predict(val_set2[:5]))
print(val_determinant_set[:5])

Epoch 1/300
40000/40000 [==============================] - 1s 28us/step - loss: 2.1496
Epoch 2/300
40000/40000 [==============================] - 1s 28us/step - loss: 2.1179
Epoch 3/300
40000/40000 [==============================] - 1s 29us/step - loss: 2.0893
Epoch 4/300
40000/40000 [==============================] - 1s 28us/step - loss: 2.0494
Epoch 5/300
40000/40000 [==============================] - 1s 29us/step - loss: 2.0058
Epoch 6/300
40000/40000 [==============================] - 1s 28us/step - loss: 1.9839
Epoch 7/300
40000/40000 [==============================] - 1s 28us/step - loss: 1.9411
Epoch 8/300
40000/40000 [==============================] - 1s 28us/step - loss: 1.8847
Epoch 9/300
40000/40000 [==============================] - 1s 29us/step - loss: 1.8588
Epoch 10/300
40000/40000 [==============================] - 1s 28us/step - loss: 1.8073
Epoch 11/300
40000/40000 [==============================] - 1s 28us/step - loss: 1.7579
Epoch 12/300
40000/40000 [===============

In [86]:
#start modeling.
#determinant modeling with fewer nodes and epochs,
#Attempting to limit over fitting.
#not working well at all
detmodel3 = models.Sequential()

detmodel3.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3),input_dim=n**2))
detmodel3.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
detmodel3.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel3.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel3.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel3.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel3.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel3.add(layers.Dense(1, activation=LeakyReLU(alpha=0.3)))
detmodel3.compile(optimizer='adam',
              loss='mean_squared_error')
detmodel3.summary()


#Simple NN model for determinant with Leaky RelU -- Overfits
#The leaky ReLU's solve the Dead ReLU problem.
detmodel3.fit(train_set2, determinant_set, epochs=100, batch_size=1000)

print("\n\nEvaluation:")
print(detmodel3.evaluate(val_set2, val_determinant_set))

print("\n\nManual Comparison on Training data")
print(detmodel3.predict(train_set2[:5]))
print(determinant_set[:5])

print("\n\nManual Comparison on Validation data")
print(detmodel3.predict(val_set2[:5]))
print(val_determinant_set[:5])

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_178 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_179 (Dense)            (None, 343)               17150     
_________________________________________________________________
dense_180 (Dense)            (None, 49)                16856     
_________________________________________________________________
dense_181 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_182 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_183 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_184 (Dense)            (None, 49)              

In [107]:
#CNN model for determinant
detCNNmodel = models.Sequential()
detCNNmodel.add(layers.Conv2D((n-2)**2, (5, 5), activation=LeakyReLU(alpha=0.3), input_shape=(n, n,1)))
# Add layers to flatten the 2D image and then normal leaky ReLU layers
detCNNmodel.add(layers.Flatten())
detCNNmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detCNNmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detCNNmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detCNNmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detCNNmodel.add(layers.Dense(1, activation=LeakyReLU(alpha=0.3)))
detCNNmodel.compile(optimizer='adam',
              loss='mean_squared_error')

detCNNmodel.summary()

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 3, 3, 25)          650       
_________________________________________________________________
flatten_15 (Flatten)         (None, 225)               0         
_________________________________________________________________
dense_200 (Dense)            (None, 49)                11074     
_________________________________________________________________
dense_201 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_202 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_203 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_204 (Dense)            (None, 1)               

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [109]:
#CNN model for determinant
detCNNmodel.fit(train_set3, determinant_set, epochs=50, batch_size=100)

print("\n\nEvaluation:")
print(detCNNmodel.evaluate(val_set3, val_determinant_set))

print("\n\nManual Comparison on Training data")
print(detCNNmodel.predict(train_set3[:5]))
print(determinant_set[:5])

print("\n\nManual Comparison on Validation data")
print(detCNNmodel.predict(val_set3[:5]))
print(val_determinant_set[:5])

Epoch 1/50
40000/40000 [==============================] - 1s 26us/step - loss: 2.3096
Epoch 2/50
40000/40000 [==============================] - 1s 26us/step - loss: 2.2967
Epoch 3/50
40000/40000 [==============================] - 1s 25us/step - loss: 2.2766
Epoch 4/50
40000/40000 [==============================] - 1s 27us/step - loss: 2.2619
Epoch 5/50
40000/40000 [==============================] - 1s 26us/step - loss: 2.2359
Epoch 6/50
40000/40000 [==============================] - 1s 26us/step - loss: 2.2179
Epoch 7/50
40000/40000 [==============================] - 1s 25us/step - loss: 2.1934
Epoch 8/50
40000/40000 [==============================] - 1s 26us/step - loss: 2.1615
Epoch 9/50
40000/40000 [==============================] - 1s 25us/step - loss: 2.1415
Epoch 10/50
40000/40000 [==============================] - 1s 27us/step - loss: 2.1103
Epoch 11/50
40000/40000 [==============================] - 1s 26us/step - loss: 2.0824
Epoch 12/50
40000/40000 [===========================

In [124]:
#start modeling.
#determinant modeling with dropout
#Attempting to limit over fitting.

detmodel4 = models.Sequential()

detmodel4.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3),input_dim=n**2))
detmodel4.add(Dropout(0.1))
detmodel4.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
detmodel4.add(Dropout(0.1))
detmodel4.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
detmodel4.add(Dropout(0.1))
detmodel4.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel4.add(Dropout(0.1))
detmodel4.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel4.add(layers.Dense(1, activation=LeakyReLU(alpha=0.3)))
detmodel4.compile(optimizer='adam',
              loss='mean_squared_error')
detmodel4.summary()


#Simple NN model for determinant with Leaky RelU -- Overfits
#The leaky ReLU's solve the Dead ReLU problem.
detmodel4.fit(train_set2, determinant_set, epochs=80, batch_size=500)

print("\n\nEvaluation:")
print(detmodel4.evaluate(val_set2, val_determinant_set))

print("\n\nManual Comparison on Training data")
print(detmodel4.predict(train_set2[:5]))
print(determinant_set[:5])

print("\n\nManual Comparison on Validation data")
print(detmodel4.predict(val_set2[:5]))
print(val_determinant_set[:5])

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_259 (Dense)            (None, 49)                2450      
_________________________________________________________________
dropout_25 (Dropout)         (None, 49)                0         
_________________________________________________________________
dense_260 (Dense)            (None, 343)               17150     
_________________________________________________________________
dropout_26 (Dropout)         (None, 343)               0         
_________________________________________________________________
dense_261 (Dense)            (None, 343)               117992    
_________________________________________________________________
dropout_27 (Dropout)         (None, 343)               0         
_________________________________________________________________
dense_262 (Dense)            (None, 49)              

In [126]:
#Eigenvalue modeling
print(eigenvalue_set.shape)

eigenmodel = models.Sequential()

# # Configure a convnet with 3 layers of convolutions and max pooling.
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Flatten())

# Add layers to flatten the 2D image and then do a 10-way classification.

eigenmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3),input_dim=n**2))
eigenmodel.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
eigenmodel.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
eigenmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
eigenmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
eigenmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
eigenmodel.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
eigenmodel.add(layers.Dense(n, activation=LeakyReLU(alpha=0.3)))
eigenmodel.compile(optimizer='adam',
              loss='mean_squared_error')
eigenmodel.summary()
eigenmodel.fit(train_set2, eigenvalue_set, epochs=80, batch_size=500)
eigenmodel.evaluate(val_set2, val_eigenvalue_set)

print("\n\nEvaluation:")
print(eigenmodel.evaluate(val_set2, val_eigenvalue_set))

print("\n\nManual Comparison on Training data")
print(eigenmodel.predict(train_set2[:5]))
print(eigenvalue_set[:5])

print("\n\nManual Comparison on Validation data")
print(eigenmodel.predict(val_set2[:5]))
print(val_eigenvalue_set[:5])

(40000, 7)
Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_271 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_272 (Dense)            (None, 343)               17150     
_________________________________________________________________
dense_273 (Dense)            (None, 343)               117992    
_________________________________________________________________
dense_274 (Dense)            (None, 49)                16856     
_________________________________________________________________
dense_275 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_276 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_277 (Dense)            (None, 49)   

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/80
40000/40000 [==============================] - 2s 41us/step - loss: 0.7795
Epoch 2/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7647
Epoch 3/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7627
Epoch 4/80
40000/40000 [==============================] - 1s 35us/step - loss: 0.7610
Epoch 5/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7598
Epoch 6/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7580
Epoch 7/80
40000/40000 [==============================] - 1s 37us/step - loss: 0.7559
Epoch 8/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7538
Epoch 9/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7506
Epoch 10/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7481
Epoch 11/80
40000/40000 [==============================] - 1s 36us/step - loss: 0.7452
Epoch 12/80
40000/40000 [===========================